In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from google.colab import drive
from tqdm.notebook import tqdm
from keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import skipgrams
drive.mount('/content/drive')

Mounted at /content/drive


Data Augmentation

In [ ]:
raw_df=pd.read_csv('/content/drive/My Drive/24787 Group Project  /Data/winemag-data-130k-v2.csv',index_col=0)
raw_data=raw_df[['description']].to_numpy()[0:-1]

In [ ]:
#tokenize the document
raw_word_db = []
raw_word_temp=[]
for batch in tqdm(range(int(10))):
    ind_start=batch*raw_data.shape[0]/10
    ind_end=ind_start+raw_data.shape[0]/10
    temp = raw_data[int(ind_start):int(ind_end)]
    raw_word_temp=[]
    for X0 in temp:
        results=text_to_word_sequence(X0[0])
        raw_word_temp=raw_word_temp+results
    raw_word_db=raw_word_db+raw_word_temp

In [ ]:
raw_word_db_Ndup=set(raw_word_db) #remove all duplicate words
word2int={} 
int2word={}
word2int['<pad>'] = 0 # add a padding token 
int2word[0] = '<pad>'
vocab_size=len(raw_word_db_Ndup)

for i, word in enumerate(raw_word_db_Ndup):
    word2int[word]=i+1
    int2word[i+1]=word

In [ ]:
sentences_num=[]
for i,X0 in enumerate(raw_data):
    temp_sentences=X0[0].split('.')
    temp_sentences=list(filter(None, temp_sentences))
    for i, sentence in enumerate(temp_sentences):
        tokens = text_to_word_sequence(sentence)
        temp_mem=[word2int[word] for word in tokens]
        sentences_num.append(temp_mem)

Generate training data Positive Skip-gram

In [ ]:
window_size = 3
X=np.zeros((1,2))
num_ls=np.array([35645,35645,35645,35645,35645,35645,35645,35645,35645,len(sentences_num)-35645-8])
for i,num in tqdm(enumerate(num_ls)):
   if i != 9:
      ind_start = i*num
      ind_end =  (i+1)*num
   else:
      ind_start = num
      ind_end = -1
   
   temp_mem = np.zeros((1,2))
   temp_sentences=sentences_num[ind_start:ind_end]

   for sentence in tqdm(temp_sentences):
         if len(sentence)!=1 and len(sentence)!=0:
            positive_skip_grams, _ = skipgrams(
                     sentence, 
                     vocabulary_size=len(word2int),
                     window_size=window_size,
                     negative_samples=0)
            temp_mem=np.concatenate([temp_mem,np.array(positive_skip_grams)],axis=0)
   X=np.concatenate([X,temp_mem[1:,:]],axis=0)

In [ ]:
X_df=pd.DataFrame(data=X[1:,:],columns=['Target','Context']).astype('int64')
IW2_df=pd.DataFrame.from_dict(int2word, orient='index',columns=['Vocab'])
X_df.to_csv('/content/drive/My Drive/24787 Group Project  /Data/X_data.csv',index=False)
IW2_df.to_csv('/content/drive/My Drive/24787 Group Project  /Data/Dictionary_HG.csv',index=False)

Negative Sampling

In [ ]:
def neg_samp(context_word,num_ns, seed):
    context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
    negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class, # class that should be sampled as 'positive'
          num_true=1, # each positive skip-gram has 1 positive context class
          num_sampled=num_ns, # number of negative context words to sample
          unique=True, # all the negative samples should be unique
          range_max=len(word2int), # pick index of the samples from [0, vocab_size]
          seed=seed, # seed for reproducibility
          name="negative_sampling" # name of this operation
    )
    return negative_sampling_candidates.numpy()

In [ ]:
num_ns=4
seed=10
tqdm.pandas()
X_df['Negative Sample']=X_df.progress_apply(lambda x : neg_samp(x['Context'],num_ns,seed),axis=1)
for num in range(num_ns):
    X_df['Negative Sample {}'.format(int(num)+1)] = X_df['Negative Sample'].progress_apply(lambda x : x[num]) 

Buffered data was truncated after reaching the output size limit.

In [ ]:
X_df.to_csv('/content/drive/My Drive/24787 Group Project  /Data/X_data.csv',index=False)